<a href="https://colab.research.google.com/github/vkrisvasan/llamaKV/blob/main/llama31groqneo4jKV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#https://console.groq.com/keys - register and get GROQ_API_KEY key
#https://console.groq.com/docs/libraries - Groq documentation
#https://docs.llamaindex.ai/en/stable/examples/cookbooks/GraphRAG_v1/
#https://github.com/projectwilsen/KnowledgeGraphLLM/blob/main/tutorial/2_convert_any_text_into_a_kg/notebook.ipynb
#https://docs.llamaindex.ai/en/stable/examples/cookbooks/llama3_cookbook_groq/
#https://www.youtube.com/watch?v=ky8LQE-82xs
#https://www.youtube.com/watch?v=KMXQ4SVLwmo

#https://docs.llamaindex.ai/en/stable/examples/property_graph/property_graph_neo4j/
#https://docs.llamaindex.ai/en/stable/understanding/loading/loading/
#https://console.neo4j.io/

!pip install groq
!pip install llama-index
!pip install neo4j
!pip install llama-index
!pip install llama-index-graph-stores-neo4j

In [ ]:
from groq import Groq

In [ ]:
!mkdir -p 'data/receipekv1'
!wget 'https://gutenberg.org/cache/epub/65061/pg65061.txt' -O 'data/receipekv1/receipekv11.txt'

--2024-08-03 14:41:24--  https://gutenberg.org/cache/epub/65061/pg65061.txt
Resolving gutenberg.org (gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to gutenberg.org (gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1398945 (1.3M) [text/plain]
Saving to: ‘data/receipekv1/receipekv11.txt’

data/receipekv1/rec 100%[===================>]   1.33M  3.58MB/s    in 0.4s    

2024-08-03 14:41:25 (3.58 MB/s) - ‘data/receipekv1/receipekv11.txt’ saved [1398945/1398945]



In [ ]:

from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader("./data/receipekv1/").load_data()

In [ ]:
from llama_index.core import PropertyGraphIndex

In [ ]:
import os
import getpass
credential_names = ["GROQ_API_KEY","NEO4J_CONNECTION_URL","NEO4J_USER","NEO4J_PASSWORD"]
for credential in credential_names:
  if credential not in os.environ:
    os.environ[credential]=getpass.getpass("Provide your..." + credential)

In [ ]:

from llama_index.graph_stores.neo4j import Neo4jPropertyGraphStore

# Note: used to be `Neo4jPGStore`
graph_store = Neo4jPropertyGraphStore(
    username=os.environ["NEO4J_USER"],
    password=os.environ["NEO4J_PASSWORD"],
    url=os.environ["NEO4J_CONNECTION_URL"],
)

ERROR:neo4j.pool:Unable to retrieve routing information


ServiceUnavailable: Unable to retrieve routing information

In [ ]:
client = Groq()

In [ ]:
def generate_content(prompt):
  reponse = client.chat.completions.create(
    model="llama-3.1-8b-instant",
    messages=[
      {"role": "system", "content": "You are a helpful assistant."},
      {"role": "user", "content": prompt}
    ]
  )
  return reponse.choices[0].message.content

In [ ]:
output = generate_content("how to covert a pdf to a graph db")
print(output)

Converting a PDF to a Graph Database involves several steps:

1. **Text Extraction**: First, you need to extract the text from the PDF file. You can use tools like `pdfplumber`, `pdfminer`, or `Tesseract OCR` to extract the text.
2. **Tokenization**: Split the extracted text into individual words or tokens.
3. **Named Entity Recognition (NER)**: Identify named entities (e.g., names, locations, organizations) in the text.
4. **Relationship Extraction**: Identify relationships between entities in the text.
5. **Graph Construction**: Construct a graph using the extracted entities and relationships.

Here's a high-level example of how you can perform these steps using Python:

**Step 1: Text Extraction**

Use `pdfplumber` to extract the text from the PDF file:
```python
import pdfplumber

with pdfplumber.open("example.pdf") as pdf:
    text = ""
    for page in pdf.pages:
        text += page.extract_text()
```
**Step 2: Tokenization**

Split the extracted text into individual words or tok

In [ ]:
output = generate_content("with example can you explain SOLID design principle")
print(output)

The SOLID design principles!

**What is SOLID?**

SOLID is an acronym that stands for five design principles of object-oriented programming (OOP) that aim to promote maintainable, flexible, and scalable software design. These principles help developers create more robust, reusable, and easier-to-maintain code by:

1. Separating concerns
2. Reducing coupling
3. Increasing cohesion
4. Improving flexibility
5. Encouraging modularity

Here's a breakdown of each principle with examples:

### S - Single Responsibility Principle (SRP)

**One class should have one reason to change**

Meaning: A class should have a single responsibility or reason to change.

**Example:**

Bad code:
```python
class OrderProcessor:
    def process_order(self):
        # Process order
        pass

    def send_email(self):
        # Send email notification
        pass
```
Good code:
```python
class OrderProcessor:
    def process_order(self):
        # Process order
        pass

class EmailNotifier:
    def sen